In [1]:
from pathlib import Path
import numpy as np
import logging

import pyvista as pv
pv.set_jupyter_backend("pythreejs")

from project_heart.enums import *
from project_heart.lv import LV

In [2]:
directory = Path("C:/Users/igorp/OneDrive - University of South Florida/Igor/Documents/Paper 1/stress_strain_plots/ref_files")
# filename = "57.32_40.94_10.08.2022_LVRNN_IDEAL_V4"
filename = "62.71_68.65_10.08.2022_LVRNN_TYPEA_V4"

In [3]:
from project_heart.examples import get_lv_ideal, get_lv_typeA
# lv = get_lv_ideal(str(directory/filename) + ".xplt", statesfile=None)
lv = get_lv_typeA(str(directory/filename) + ".xplt", statesfile=None)

INFO:LV_RegionIdentifier.identify_regions_typeA:Identifying regions from 'type A' geometry.
INFO:LV_RegionIdentifier.identify_endo_epi_base_borders_from_nodesets:len(endo_base): 329
INFO:LV_RegionIdentifier.identify_endo_epi_base_borders_from_nodesets:len(epi_base): 369


In [4]:
import logging

names = ["subapex", "apex", "superapex", "submid", "mid", "supermid", "subbase", "base", "superbase"]
for i, a in enumerate(np.linspace(0.1, 0.90, len(names), endpoint=True)):
    
    _ = lv.create_speckles(
        collection="circ-6",
        group="wall",
        name=names[i],
        from_nodeset=None, # use all spks
        d=2.25,
        k=a,
        normal_to=[0.0, 0.0, 1.0],
        n_subsets=6,
        subsets_criteria="angles2",
        # subsets_vector_ref=[1.0,0.0,0.0],
        n_clusters=5,
        cluster_criteria="radius2",
        t=0.0,
        log_level=logging.WARN,
    )

In [5]:
spks = lv.get_speckles(spk_collection="circ-6", spk_group="wall")
subsets_names = ["endo", "endo-mid", "mid", "mid-epi", "epi"]
lv.split_spks_by_clusters(spks, rename_subsets=subsets_names,switch_subset_group=True)
spks_split = lv.get_speckles(spk_collection="circ-6_split")

In [6]:
spks_split[0].elem_ids

array([75171, 75233, 76331, 76448, 76449, 76450, 76451, 76452, 76453,
       76454, 76455, 76458, 76707, 76864, 76865, 76867, 76868, 76869,
       76870, 76871, 76872, 76873, 76874, 76875, 76878, 77344, 77345,
       77346, 77347, 77348, 77349, 77350, 77352, 77504, 77506, 77664,
       77665, 77666, 77667, 77668, 77669, 77670, 77671, 77675, 77760,
       77862, 77896, 78208, 78209, 78210, 78211, 78212, 78213, 78214,
       78215, 78216, 78217, 78218, 78219, 78240, 78241, 78242, 78243,
       78244, 78246, 78304, 78305, 78306, 78307, 78308, 78309, 78310,
       78311, 78529, 78531], dtype=int64)

In [ ]:
spks_mid = lv.get_speckles(spk_collection="circ-6_split", spk_name="mid", spk_group="mid-epi")
lv.plot_speckles(spks_mid, show_clusters=False)

In [7]:
spks_base = lv.get_speckles(spk_collection="circ-6_split", spk_name="base", spk_group="endo")
lv.plot_speckles(spks_base, show_clusters=False)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, 1…

In [8]:
lv.compute_stress(spks_split, reduce_by=["group_name"], principal=True, use_axis=None, cylindrical=False, effective=False, log_level=logging.WARN)

INFO:LV.BaseMetricsComputations:Computing metric 'LV_STATES.PRINCIPAL_STRESS' with 'effective' set to 'False'


In [9]:
lv.compute_strain(spks_split, reduce_by=["group_name"],  principal=True, use_axis=None, cylindrical=False, effective=False, log_level=logging.WARN)

In [10]:
_, stress_data = lv.get_metric_as_df("principal_stress")
_, strain_data = lv.get_metric_as_df("principal_strain")

In [12]:
stress_df = stress_data["group_name"]
stress_df = stress_df[stress_df["timesteps"] > 0.1] - 0.1
stress_df.to_csv(str(directory/filename) + "_stress.csv")

In [ ]:
strain_df = strain_data["group_name"]
strain_df = strain_df[strain_df["timesteps"] > 0.1] - 0.1
strain_df.to_csv(str(directory/filename) + "_strain.csv")